# 從零開始構建VGG網絡來學習Keras

從零開始使用Keras來構建在ILSVRC-2014 (ImageNet competition)競賽中獲的第一名的VGG網路結構。

![vgg16](https://www.cs.toronto.edu/~frossard/post/vgg16/vgg16.png)

![vgg network structure](https://cdn-images-1.medium.com/max/1600/1*FRd9fDM1TXThW2V8ylL7VQ.png)


![vgg 2.1](https://cdn-images-1.medium.com/max/1600/1*DwsWBmGCI7qL9ei7n_SdXA.png)

* input size 224 x 224
* kernel3 x 3
* stride是1
*padding是1
* 池化層是2×2且stride為2
* 有兩個完全連接層，每層4096個神經元
* 最後一層是具有1000個神經元的`softmax`分類層
* activation是`ReLU`

In [ ]:
# 這個Jupyter Notebook的環境
import platform
import tensorflow
from tensorflow import keras
print("Platform: {}".format(platform.platform()))
print("Tensorflow version: {}".format(tensorflow.__version__))
print("Keras version: {}".format(keras.__version__))

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from IPython.display import Image

Platform: Linux-5.15.107+-x86_64-with-glibc2.31
Tensorflow version: 2.12.0
Keras version: 2.12.0


## 創建模型 (Functaional API)


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Activation, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPool2D

# 定義輸入
input_shape = (224, 224, 3) # RGB影像224x224 (width, height, channel)

# 輸入層
img_input = Input(shape=input_shape, name='img_input')

# 第1個卷積區塊 (block1)
x = Conv2D(64, (3, 3), padding='same', activation='relu', name='block1_conv1')(img_input)
x = Conv2D(64, (3, 3), padding='same', activation='relu', name='block1_conv2')(x)
x = MaxPool2D((2, 2), strides=(2, 2), name='block1_pool')(x)

# 第2個卷積區塊 (block2)
x = Conv2D(128, (3, 3), padding='same', activation='relu', name='block2_conv1')(x)
x = Conv2D(128, (3, 3), padding='same', activation='relu', name='block2_conv2')(x)
x = MaxPool2D((2, 2), strides=(2, 2), name='block2_pool')(x)

# 第3個卷積區塊 (block3)
x = Conv2D(256, (3, 3), padding='same', activation='relu', name='block3_conv1')(x)
x = Conv2D(256, (3, 3), padding='same', activation='relu', name='block3_conv2')(x)
x = Conv2D(256, (3, 3), padding='same', activation='relu', name='block3_conv3')(x)
x = MaxPool2D((2, 2), strides=(2, 2), name='block3_pool')(x)

# 第4個卷積區塊 (block4)
x = Conv2D(512, (3, 3), padding='same', activation='relu', name='block4_conv1')(x)
x = Conv2D(512, (3, 3), padding='same', activation='relu', name='block4_conv2')(x)
x = Conv2D(512, (3, 3), padding='same', activation='relu', name='block4_conv3')(x)
x = MaxPool2D((2, 2), strides=(2, 2), name='block4_pool')(x)

# 第5個卷積區塊 (block5)
x = Conv2D(512, (3, 3), padding='same', activation='relu', name='block5_conv1')(x)
x = Conv2D(512, (3, 3), padding='same', activation='relu', name='block5_conv2')(x)
x = Conv2D(512, (3, 3), padding='same', activation='relu', name='block5_conv3')(x)
x = MaxPool2D((2, 2), strides=(2, 2), name='block5_pool')(x)

# 前饋全連接區塊
x = Flatten(name='flatten')(x)
x = Dense(4096, activation='relu', name='fc1')(x)
x = Dense(4096, activation='relu', name='fc2')(x)
x = Dense(1000, activation='softmax')(x)

# 產生模型
model2 = Model(inputs=img_input, outputs=x)

# 打印網絡結構
model2.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 img_input (InputLayer)      [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

### 確認模型訓練的參數總數
構建的模型138,357,544參數的確符合論文提及的138百萬的訓練參數。

![model-parameters](https://cdn-images-1.medium.com/max/1600/1*cY2yGoETl0jBrtky8PGysg.png)

## 模型訓練


>On a system equipped with four NVIDIA Titan Black GPUs, training a single net took 2–3 weeks depending on the architecture.

詳請請見: [keras.io](https://keras.io/applications/#vgg16)

參考:
* [Learning Keras by Implementing the VGG Network From Scratch](https://hackernoon.com/learning-keras-by-implementing-vgg16-from-scratch-d036733f2d5/)
* [Keras官網](http://keras.io/)

*https://github.com/erhwenkuo